In [74]:
import pandas as pd
import json
import numpy as np
import os
import glob
import re
import urllib
from bs4 import BeautifulSoup
from pathlib import Path


In [75]:
#def ig_unknown(file, hl_min, hl_max):
#    return "Feature not implemented yet. Write an issue to GitHub."

In [76]:
#def ig_isotope(measured_df, mat_list, hl_min, hl_max, name):
#    precursores = mat_list.split(',')
#    isotopes = []
#    with open("database/irr_products.json") as f:
#        irradiation_products = json.load(f)
#    
#    for precursor in precursores:
#        try:
#            isotopes += irradiation_products[precursor]
#        
#        except:
#            irradiation_products[precursor] = load_irr_prod(precursor)
#            with open("database/irr_products.json", 'w') as json_out:
#                json.dump(irradiation_products, json_out)
#
#    # get unique only
#    isotopes = list(set(isotopes))
#
#    db_parameters = ["half-life", "sigm_half-life", "lambda", "sigm_lambda", "isotope", "Ig", "sigm_Ig", "E_tab", "sigm_E_tab"]#
#
#    measured_df[[db_parameters]] = np.nan*np.ones(shape=(measured_df.shape[0], len(db_parameters)))
#
#    for isotope in isotopes:
#        measured_df = add_db_info(measured_df, isotope)
#    
#    measured_df.to_csv(f"out/{name}wDBinfo.csv")

In [77]:
#def ig_nat_element(file, mat_list, hl_min, hl_max):
#    return "Feature not implemented yet. Write an issue to GitHub."

In [78]:
#def ig_element_spec(file, mat_list, hl_min, hl_max):
#    return "Feature not implemented yet. Write an issue to GitHub."

In [79]:
#def ig_material(file, mat_list, hl_min, hl_max):
#    return "Feature not implemented yet. Write an issue to GitHub."

In [80]:
def append_Igamma_dir(parsed_dir):
    if not os.path.isdir(parsed_dir):
        raise Exception(f"Directory '{parsed_dir}' not found. Consider creating it and moving parsed files there.")

    for parsed_file in glob.iglob(f"{parsed_dir}/*.csv"):
        append_Igamma(parsed_file)

In [81]:
def append_Igamma(file):
    print(file)
    A, element = re.split(r'(\d+)(\w+)', file)[-3:-1]
    element = element.split("_")[0]
    print(A)
    print(element)

    #Path("./ig_db").mkdir(parents=True, exist_ok=True)
    ig_all_df = pd.read_parquet("ig_db/ig_all.pq")
    print(f"A is {A} and element is {element}")
    ig_df = ig_all_df.loc[f"{A}{element}"]

    #if os.path.isfile(f"ig_db/{A}{element}.csv"):
    #    ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
    #    print("Reading the csv with gammas.")
    #else:
    #    Path("./downloads").mkdir(parents=True, exist_ok=True)
    #    if os.path.isfile(f"downloads/{A}{element}.html"):
    #        extract_Igamma(A, element)
    #        ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
    #        print("Reading the csv with gammas.")
    #    else:
    #        download(A, element)
    #        extract_Igamma(A, element)
    #        ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
    #        print("Reading the csv with gammas.")
    
    parsed_df = pd.read_csv(file, index_col=0)
    joined_df = add_Ig(parsed_df, ig_df)
    
    Path("./with_Ig").mkdir(parents=True, exist_ok=True)
    joined_df.to_csv(f"with_Ig/{A}{element}.csv")


In [82]:
#df = pd.read_parquet("ig_db/ig_all.pq")

In [83]:
#df.loc["152Eu"]

In [84]:
def add_Ig(df, ig, ig_thr):
    e_col = []
    ig_col = []

    a = np.empty((df.shape[0], 2))
    a[:] = np.nan
    df[["E_tab", "Ig"]] = a
    # UGLY, never for-loop in pandas
    for row_no in range(df.shape[0]):
        #suitable_lines = ig[(df["Energy"] - df["FWHM"]).iloc[row_no] > ig["E_tab"] + ig["sigm_E"]]
        suitable_lines = ig[((df["Energy"] - df["FWHM"]).iloc[row_no] < ig["E_tab"] + ig["sigm_E"]) & ((df["Energy"] + df["FWHM"]).iloc[row_no] > ig["E_tab"] - ig["sigm_E"]) & ig["Ig"] > 0.01*ig_thr]
        #for ig_row_no in range(ig.shape[0]):
        #    if (df["Energy"] - df["FWHM"]).iloc[row_no] > ig["E_tab"].iloc[ig_row_no] + ig["sigm_E"].iloc[ig_row_no]:
        #        continue
        #    elif (df["Energy"] + df["FWHM"]).iloc[row_no] < ig["E_tab"].iloc[ig_row_no] - ig["sigm_E"].iloc[ig_row_no]:
        #        break
        #    else:
        #        df.loc[row_no, ["E_tab", "sigm_E_tab", "Ig", "sigm_Ig"]] = ig.loc[ig_row_no]
    print("Ig added")
    return df

In [89]:
ig = pd.read_parquet("ig_db/ig_all.pq")
df = pd.read_csv("parsed_reports/152Eu_g80.csv", index_col=0)

In [101]:
ig[((df["Energy"] - df["FWHM"]).iloc[0] < ig["E_tab"] + ig["sigm_E"]) & ((df["Energy"] + df["FWHM"]).iloc[0] > ig["E_tab"] - ig["sigm_E"]) & ig["Ig"] > 0.0001]

,E_tab,sigm_E,Ig,sigm_Ig
Isotope,,,,
77Kr,24.200001,0.50000,0.000360,6.000000e-05
147Cs,24.500000,0.30000,0.000290,1.400000e-04
155Eu,24.559999,0.30000,0.000080,8.000000e-05
103mRu,24.600000,0.70000,0.001300,NaN
227Ra,24.700001,0.20000,0.000340,NaN
231Pa,24.700001,0.20000,0.000050,NaN
255Fm,24.823999,0.01500,0.000900,2.000000e-04
58mCo,24.889000,0.02100,0.000389,1.200000e-05
228Pa,24.900000,0.20000,0.000020,1.000000e-05


In [98]:
ig[((df["Energy"] - df["FWHM"]).iloc[0] < ig["E_tab"] + ig["sigm_E"])]

,E_tab,sigm_E,Ig,sigm_Ig
Isotope,,,,
240mNp,20.000000,15.0,0.00000,NaN
77Kr,24.200001,0.5,0.00036,0.00006
147Cs,24.500000,0.3,0.00029,0.00014
227Ac,24.500000,0.2,NaN,NaN
155Eu,24.559999,0.3,0.00008,0.00008
...,...,...,...,...
27310,11350.000000,60.0,0.50000,NaN
27712,11450.000000,20.0,0.50000,NaN
26710,11600.000000,1000.0,1.00000,NaN


In [99]:
ig[((df["Energy"] + df["FWHM"]).iloc[0] > ig["E_tab"] - ig["sigm_E"])]

,E_tab,sigm_E,Ig,sigm_Ig
Isotope,,,,
235Pa,0.076800,0.0005,NaN,NaN
235mU,0.076800,0.0005,NaN,NaN
239Pu,0.076800,0.0005,NaN,NaN
229U,0.220000,0.0500,NaN,NaN
255Fm,0.570000,0.0300,NaN,NaN
...,...,...,...,...
221Rn,26.000000,0.1000,NaN,NaN
157Yb,26.049999,0.1700,0.05400,0.00500
185mHg,26.100000,0.5000,0.00025,0.00006


In [85]:
# NO need for this now, deprecated

#def getZ(element):
#    element_Z ={
#        "Eu": 63,
#        "U": 92,
#        "Pu": 94
#    }
#    return element_Z[element]

In [86]:
# No need for this now, deprecated

#def download(A, element):
#    url = f"http://nucleardata.nuclear.lu.se/toi/nuclide.asp?iZA={getZ(element)}0{A}"
#    urllib.request.urlretrieve (url, f"downloads/{A}{element}.html")
#    print(f"File {A}{element}.html downloaded.")

In [87]:
#def extract_Igamma(A, element):
#    html_file = open(f"downloads/{A}{element}.html", "r")
#    soup = BeautifulSoup(html_file.read(), 'lxml')
    
#    gammas_table = soup.find_all("table")[4]
#    gammas_rows = gammas_table.find_all('tr')[3:-1]
#    energy = []
#    sigm_energy = []
#    i = []
#    sigm_i = []

#    for row in gammas_rows:
#        cells = row.find_all('td')
        
#        e_val = cells[0].get_text(strip=True)
#        i_val = cells[1].get_text(strip=True)
#        try:
#            ig_val = float(i_val[:-1])
#            sigm_ig_val = float(i_val[-1])
#        except:
#            ig_val = float('NaN')
#            sigm_ig_val = float('NaN')
       
#        energy.append(float(e_val[:-1]))
#        sigm_energy.append(int(e_val[-1]))
#        i.append(ig_val)
        
#        sigm_i.append(sigm_ig_val)


#    df_dict = {
#        "E_tab": energy,
#        "sigm_E_tab": sigm_energy, 
#        "Ig": i,
#        "sigm_Ig": sigm_i
#        }
#    df = pd.DataFrame(df_dict)
#    df_name = f'ig_db/{A}{element}.csv'
#    df.to_csv(df_name)
   
#    print(f"Ig extracted from file 'downloads/{A}{element}.html' into '{df_name}'.")

In [88]:
append_Igamma_dir("parsed_reports")

parsed_reports/152Eu_g80.csv
152
Eu
A is 152 and element is Eu


KeyError: 4